Цель — научиться вычислять эмбеддинги текстов.
Необходимо сделать модель машинного обучения, вычисляющую эмбеддинги текста (векторное представление фиксированной длины), используя готовые наработки либо обучив её самостоятельно. Вид векторного представления может быть произвольным, но каждый из эмбеддингов не должен содержать более 1000 чисел.

После чего нужно по датасету вопросов Quora Question Pairs (https://www.kaggle.com/c/quora-question-pairs/) обучить модель, возвращающую вероятность похожести пар вопросов только по паре их эмбеддингов. Модель должна выдавать на тестовой выборке в публичном лидерборде kaggle logloss не больше 0,5.

### 1. Вычисление эмбеддингов

Использую готовую библиотеку для вычисления эмбеддингов

In [8]:
from collections import Counter
import numpy as np 
import pandas as pd 
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

In [6]:
model = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [7]:
model.wv.most_similar("pelosi",topn=10)

C:\Users\Alibekk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('clegg', 0.9653651118278503),
 ('miliband', 0.9515050053596497),
 ('bachmann', 0.9484400749206543),
 ('mcconnell', 0.9416399002075195),
 ('carney', 0.934025764465332),
 ('coulter', 0.9311323761940002),
 ('boehner', 0.9286302328109741),
 ('santorum', 0.9269059300422668),
 ('farage', 0.919365406036377),
 ('mourdock', 0.9186689853668213)]

In [ ]:
#(Не стал запускать код - долго обучает, по времени не успел()
corpus = api.load('text8')

# обучение модели word2vec на корпусе текстов
model_text8 = Word2Vec(corpus,iter=10, size=1000, window=10, min_count=2, workers=10)

In [ ]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

### 2. Quora Question Pairs

In [23]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [24]:
train_qs = pd.Series(df_train['question1'].tolist() + df_train['question2'].tolist()).astype(str)
test_qs = pd.Series(df_test['question1'].tolist() + df_test['question2'].tolist()).astype(str)

In [30]:
dist_train = train_qs.apply(lambda x: len(x.split(' ')))
dist_test = test_qs.apply(lambda x: len(x.split(' ')))

Удаление стопслов и начальный анализ

In [36]:
stops = set(stopwords.words("english"))

def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

### tf-idf

In [37]:
def get_weight(count, eps=10000, min_count=2):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)

eps = 5000 
words = (" ".join(train_qs)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}

In [38]:
def tfidf_word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R

In [40]:
tfidf_train_word_match = df_train.apply(tfidf_word_match_share, axis=1, raw=True)

C:\Users\Alibekk\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [41]:
print('Original AUC:', roc_auc_score(df_train['is_duplicate'], train_word_match))
print(' TFIDF AUC:', roc_auc_score(df_train['is_duplicate'], tfidf_train_word_match.fillna(0)))

Original AUC: 0.7804327049353577
 TFIDF AUC: 0.7704802292218704


#### Балансировка данных

In [42]:
x_train = pd.DataFrame()
x_test = pd.DataFrame()
x_train['word_match'] = train_word_match
x_train['tfidf_word_match'] = tfidf_train_word_match
x_test['word_match'] = df_test.apply(word_match_share, axis=1, raw=True)
x_test['tfidf_word_match'] = df_test.apply(tfidf_word_match_share, axis=1, raw=True)

y_train = df_train['is_duplicate'].values

C:\Users\Alibekk\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
C:\Users\Alibekk\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in long_scalars
  app.launch_new_instance()


In [43]:
pos_train = x_train[y_train == 1]
neg_train = x_train[y_train == 0]

# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

x_train = pd.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
del pos_train, neg_train

0.19124366100096607


In [46]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=1242)

### Применяю модель xgboost

In [48]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 500, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.682797	valid-logloss:0.683336
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.602389	valid-logloss:0.602324
[20]	train-logloss:0.545213	valid-logloss:0.545464
[30]	train-logloss:0.503348	valid-logloss:0.503943
[40]	train-logloss:0.471667	valid-logloss:0.472932
[50]	train-logloss:0.448513	valid-logloss:0.449326
[60]	train-logloss:0.430594	valid-logloss:0.431156
[70]	train-logloss:0.416344	valid-logloss:0.41716
[80]	train-logloss:0.405347	valid-logloss:0.40619
[90]	train-logloss:0.396571	valid-logloss:0.397536
[100]	train-logloss:0.389763	valid-logloss:0.39076
[110]	train-logloss:0.38428	valid-logloss:0.385381
[120]	train-logloss:0.379937	valid-logloss:0.381106
[130]	train-logloss:0.376644	valid-logloss:0.37769
[140]	train-logloss:0.374052	valid-logloss:0.37504
[150]	train-logloss:0.371656	valid-logloss:0.372877
[160]	train-logloss:0.369929	valid

In [49]:
d_test = xgb.DMatrix(x_test)
p_test = bst.predict(d_test)

sub = pd.DataFrame()
sub['test_id'] = df_test['test_id']
sub['is_duplicate'] = p_test
sub.to_csv('simple_xgb.csv', index=False)